In [1]:
import tensorflow as tf
import numpy as np
import os, sys, string, random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [2]:
def color_to_alpha(image, target_color=None):
    
    data = np.array(image)
    
    rgb = data[:,:,:3]
    
    if target_color is None:
        target_color =  [255, 255, 255]  # Original value
        
    transparent_color = [0, 0, 0, 0]
    mask = np.all(rgb == target_color, axis = -1)
    
    # change all pixels that match color to transparent_color*
    data[mask] = transparent_color
    
    return data

In [3]:
filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./random_dir/original/*.png"))

In [4]:
image_reader = tf.WholeFileReader()

In [5]:
_, image_file = image_reader.read(filename_queue)

In [6]:
image           = tf.image.decode_png(image_file, channels=4)
image_processed = tf.py_func(color_to_alpha, [image], tf.uint8)
image_processed = tf.image.encode_png(image_processed)

In [7]:
init_op_global = tf.global_variables_initializer()
init_op_local = tf.local_variables_initializer()

In [8]:
def id_generator(size=20, chars=string.ascii_uppercase + string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [11]:
out_dir = "random_dir/output"

with tf.Session() as sess:
    # Required to get the filename matching to run.
    sess.run([init_op_global, init_op_local])

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Get an image tensor and print its value.
    _image_processed = sess.run(image_processed)
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    filename = out_dir + "/" + id_generator() + ".jpg"

    with open(filename, "wb+") as f:
        f.write(_image_processed)
        f.close()

    # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)